In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)


In [5]:
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    # 여기서 output이 hidden cell이다.
    hidden_size = 2 
    # 가장 기본 RNN 구조
    # 이 안에서 LSTM으로 바꾸고 싶으면 cell만 LSTM으로 바꾸면 된다.
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    # state는 모델 상태
    # outputs
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
array([[[-0.41376624, -0.51413125]]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

In [6]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.57803905, -0.6183005 ],
        [ 0.5357774 ,  0.04099169],
        [ 0.07504235,  0.7707948 ],
        [-0.42017367,  0.9364969 ],
        [-0.4576048 ,  0.29156446]]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

In [7]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    # 그래서 shape이 (3,5,2)이다
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.09321102,  0.03219642],
        [ 0.0615468 , -0.00332276],
        [ 0.08537696,  0.11503907],
        [ 0.09739595,  0.19838296],
        [ 0.03078922,  0.12711605]],

       [[ 0.0005288 , -0.03673532],
        [-0.05999489, -0.14788014],
        [ 0.01749653,  0.04193721],
        [ 0.05634893,  0.15041941],
        [ 0.07998287,  0.22307625]],

       [[ 0.04531673,  0.11977081],
        [ 0.07343666,  0.20312035],
        [ 0.04965942,  0.2051381 ],
        [ 0.03521229,  0.1454849 ],
        [ 0.06833738,  0.19704822]]], dtype=float32)


In [23]:
with tf.variable_scope('3_batches_dynamic_length3') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    # sequence_length는 각 배치마다 시퀀스의 크기를 리스트에 저장할 수 있다.
    # 지금 5,3,4이니 hello, eol, llee만 값이 출력되고 나머지부분은 0으로 처리된다.
    # 이것이 있는 이유는 입력값이 다를 것을 대비한 것이다.
    # 예를들어 문장을 형태소별로 잘랐을 때 각 토큰의 길이가 다를 것이다.
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.19303772,  0.02679601],
        [-0.17133988, -0.06782463],
        [-0.17099826,  0.06548732],
        [-0.2181655 ,  0.14868738],
        [-0.2833136 , -0.02352209]],

       [[-0.02502428, -0.08224939],
        [-0.06298991, -0.19274323],
        [-0.0998312 ,  0.00410579],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.07655006,  0.11170713],
        [-0.14978342,  0.17590529],
        [-0.20550278,  0.04497789],
        [-0.1836722 , -0.05184217],
        [ 0.        ,  0.        ]]], dtype=float32)


In [25]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    # RNN 초기값을 0으로 줌
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.10078802,  0.13400415],
        [-0.03830677,  0.07768194],
        [-0.12962961,  0.11632847],
        [-0.18225542,  0.14156449],
        [ 0.03863295,  0.22164395]],

       [[-0.10713828, -0.03386441],
        [ 0.048637  ,  0.07024362],
        [-0.04421161,  0.09679724],
        [-0.12207574,  0.12966135],
        [-0.17579196,  0.15168872]],

       [[-0.09680256,  0.05833871],
        [-0.16274075,  0.09911373],
        [-0.17316027,  0.04666985],
        [-0.21002096, -0.00145557],
        [-0.30321562,  0.05097376]]], dtype=float32)


In [3]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  # batch, sequence_length, input_dim

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)


In [27]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[-9.57843438e-02, -7.54647478e-02, -4.60432246e-02,
         -1.16779432e-01,  1.51008800e-01],
        [-4.33588386e-01, -9.85343605e-02,  1.06190324e-01,
         -1.36943445e-01,  8.91944394e-02],
        [-6.37848675e-01, -2.08311081e-02,  3.11666608e-01,
         -1.06611289e-01,  1.82022341e-02],
        [-7.45411515e-01,  9.12511349e-02,  4.65661258e-01,
         -5.43830395e-02,  3.11620929e-03],
        [-7.99489498e-01,  1.73044086e-01,  5.59863985e-01,
         -1.26141896e-02,  5.18138520e-04]],

       [[-6.65292263e-01,  2.19574064e-01,  2.99058199e-01,
          1.93794966e-02,  8.06401586e-05],
        [-7.92220831e-01,  2.70482570e-01,  5.50214410e-01,
          3.32712904e-02,  1.40703987e-05],
        [-8.13453972e-01,  2.98774838e-01,  6.48099422e-01,
          3.96890976e-02,  2.48538458e-06],
        [-8.10372114e-01,  3.18632096e-01,  6.88364744e-01,
          4.18972857e-02,  4.36029069e-07],
        [-8.02301109e-01,  3.31396788e-01,  7.12126970e-01,
  

In [37]:
with tf.variable_scope('MultiRNNCell4') as scope:
    # Make rnn
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [128,256]]
    cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("MultiRNNCell4/rnn/transpose_1:0", shape=(3, 5, 256), dtype=float32)
array([[[ 1.50780287e-03,  5.44891518e-04, -4.35640663e-03, ...,
         -1.34728674e-04,  9.61986277e-03, -9.43821156e-04],
        [ 6.15415676e-03,  9.77632869e-03, -1.21697905e-02, ...,
          3.60852387e-03,  3.67949530e-02, -4.81868861e-03],
        [ 9.74488445e-03,  3.16993669e-02, -2.49429774e-02, ...,
          1.43170096e-02,  7.52642155e-02, -1.54824657e-02],
        [ 1.08603928e-02,  6.44593164e-02, -4.37167808e-02, ...,
          2.88909543e-02,  1.15434825e-01, -3.12048420e-02],
        [ 1.17533170e-02,  1.01916976e-01, -6.84865266e-02, ...,
          4.14157957e-02,  1.51126206e-01, -4.75034192e-02]],

       [[-4.70651034e-03,  2.16581672e-02, -2.90456619e-02, ...,
          2.32079756e-02,  3.86994407e-02, -1.40672009e-02],
        [-9.53856483e-03,  5.98175973e-02, -6.55082390e-02, ...,
          4.07730825e-02,  8.21478143e-02, -3.57319042e-02],
        [-1.06952731e-02, 

In [5]:
with tf.variable_scope('dynamic_rnn') as scope:
    cell = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True)
    # dynamic_rnn()은 deprecated됨.
    # RNN cell을 이용해 RNN 만든는 함수
    # tf.keras.layers.RNN 사용할 것
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])
   
    # lentgh 1 for batch 1, lentgh 2 for batch 2
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("dynamic_rnn_1/rnn/transpose_1:0", shape=(3, 5, 5), dtype=float32)
array([[[-1.3645709e-02,  8.2412802e-02, -4.4301571e-03, -5.0062563e-02,
         -3.6789931e-02],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00]],

       [[-6.8740512e-04,  6.2050349e-01, -7.4499876e-06, -1.0655648e-04,
         -1.6122357e-03],
        [-3.7477358e-04,  9.1176504e-01, -4.6680098e-06, -1.2979897e-04,
         -6.1426457e-04],
        [-1.7568872e-04,  9.8368806e-01, -2.1731034e-06, -1.2793619e-04,
         -1.9982994e-04],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000

In [6]:
with tf.variable_scope('bi-directional') as scope:
    # bi-directional rnn
    # 양방향 RNN
    # forward
    # tf.keras.layers.LSTMCell로 대체될 것.
    cell_fw = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True)
    # backward
    cell_bw = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

(   array([[[ 6.25624731e-02, -9.57042277e-02,  5.77053279e-02,
         -3.16355713e-02, -8.33941251e-02],
        [ 7.00005516e-02, -1.19264096e-01,  1.21350558e-02,
         -1.39794953e-03, -1.33683130e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00]],

       [[ 1.29859662e-03, -1.88081991e-02,  3.50208836e-07,
          4.01564158e-14, -1.39227789e-03],
        [ 1.07748632e-03, -1.14402566e-02,  3.83073981e-08,
          1.48296178e-15, -5.46327617e-04],
        [ 6.68946130e-04, -6.94234250e-03,  3.25294214e-09,
          4.98262003e-17, -1.70079409e-04],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00

In [8]:
# flatten based softmax
hidden_size=3
sequence_length=5
batch_size=3
num_classes=5

print('== 처음 데이터 ==')
pp.pprint(x_data) # hidden_size=3, sequence_length=4, batch_size=2

print('\n== 배치를 합친 데이터(flatten) ==')
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

print('\n== softmax를 거친 데이터 ==')
softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
pp.pprint(outputs)

== 처음 데이터 ==
array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.],
       [12., 13., 14.],
       [15., 16., 17.],
       [18., 19., 20.],
       [21., 22., 23.],
       [24., 25., 26.],
       [27., 28., 29.],
       [30., 31., 32.],
       [33., 34., 35.],
       [36., 37., 38.],
       [39., 40., 41.],
       [42., 43., 44.]], dtype=float32)

== 배치를 합친 데이터(flatten) ==
array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.],
       [12., 13., 14.],
       [15., 16., 17.],
       [18., 19., 20.],
       [21., 22., 23.],
       [24., 25., 26.],
       [27., 28., 29.],
       [30., 31., 32.],
       [33., 34., 35.],
       [36., 37., 38.],
       [39., 40., 41.],
       [42., 43., 44.]], dtype=float32)

== softmax를 거친 데이터 ==
array([[[  25.,   28.,   31.,   34.,   37.],
        [  70.,   82.,   94.,  106.,  118.],
        [ 115.,  136.,  157.,  178.,  199.],
        [ 160.,  190.,  220.,  250.,  280.],
     

In [16]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

# Weighted cross-entropy loss for a sequence of logits.
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.596759


In [10]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)

prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction4 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "\nLoss2: ", sequence_loss2.eval(),
      "\nLoss3: ", sequence_loss3.eval(),
      "\nLoss4: ", sequence_loss4.eval())

Loss1:  0.5130153 
Loss2:  0.3711007 
Loss3:  1.3132616 
Loss4:  1.3132616
